In [1]:

import sys
sys.path.append("../build/mechsystem")
from mass_spring import *

from pythreejs import *

mss = MassSpringSystem3d()
mss.gravity = (0, 0, -9.81)

In [2]:
n = 5                       # nombre de masses dans la chaîne
spacing = 1.0               # distance entre les masses
k = 2000                  # raideur des ressorts
m = 1                       # masse

masses_ids = []

# 1) Ajouter le point fixe
f1 = mss.add(Fix((0, 0, 0)))

# 2) Ajouter les masses le long de l'axe X
for i in range(n):
    pos = (spacing * (i+1), 0, 0)   # Masses en ligne
    masses_ids.append(mss.add(Mass(m, pos)))

# 3) Connecter fix -> masse 1
mss.add(Spring(1, k, (f1, masses_ids[0])))

# 4) Connecter masse[i] -> masse[i+1]
for i in range(n - 1):
    mss.add(Spring(1, k, (masses_ids[i], masses_ids[i+1])))

In [3]:
masses = []
for m in mss.masses:
    masses.append(
        Mesh(SphereBufferGeometry(0.2, 16, 16),
             MeshStandardMaterial(color='red'),
             position=m.pos)) 

fixes = []
for f in mss.fixes:
    fixes.append(
        Mesh(SphereBufferGeometry(0.2, 32, 16),
             MeshStandardMaterial(color='blue'),
             position=f.pos)) 

springpos = []
for s in mss.springs:
    pA = mss[s.connectors[0]].pos
    pB = mss[s.connectors[1]].pos
    springpos.append ([ pA, pB ] ) 

springgeo = LineSegmentsGeometry(positions=springpos)
m2 = LineMaterial(linewidth=3, color='cyan')
springs = LineSegments2(springgeo, m2)    

axes = AxesHelper(1)

In [4]:
view_width = 600
view_height = 400

camera = PerspectiveCamera( position=[10, 6, 10], aspect=view_width/view_height)
key_light = DirectionalLight(position=[0, 10, 10])
ambient_light = AmbientLight()

scene = Scene(children=[*masses, *fixes, springs, axes, camera, key_light, ambient_light])
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller],
                    width=view_width, height=view_height)

renderer

Renderer(camera=PerspectiveCamera(aspect=1.5, position=(10.0, 6.0, 10.0), projectionMatrix=(1.0, 0.0, 0.0, 0.0…

In [5]:
from time import sleep
for i in range(10000):
    mss.simulate (0.005, 20)
    for m,mvis in zip(mss.masses, masses):
        mvis.position = (m.pos[0], m.pos[1], m.pos[2])

    springpos = []
    for s in mss.springs:
        pA = mss[s.connectors[0]].pos
        pB = mss[s.connectors[1]].pos
        springpos.append ([ pA, pB ]) 
    springs.geometry = LineSegmentsGeometry(positions=springpos)
    sleep(0.01)